# Visualize saved outputs

In [1]:
from pylab import *
%matplotlib widget
from matplotlib.legend_handler import HandlerTuple

import numpy as np

ModuleNotFoundError: No module named 'pylab'

#### Data Loader

In [2]:
def load_data(output_folder):
    bounds = np.loadtxt(output_folder + 'bounds.txt')
    three_entropies = np.loadtxt(output_folder + 'three_entropies.txt')

    try:
        log_likelihood = np.loadtxt(output_folder + 'log_likelihood.txt')
        if len(log_likelihood) == 0:
            log_likelihood = None
    except:
        log_likelihood = None

    try:
        test_log_likelihood = np.loadtxt(output_folder + 'test_log_likelihood.txt')
        if len(test_log_likelihood) == 0:
            test_log_likelihood = None
    except:
        test_log_likelihood = None
    
    try:
        ground_truth_ll = [np.loadtxt(output_folder + 'ground_truth_ll.txt')]
    except:
        ground_truth_ll = None
    return bounds, three_entropies, log_likelihood, test_log_likelihood, ground_truth_ll

# Paper Plots

## Fig. 1a: Linear VAE on PCA

In [3]:
batch_size = 2000
lr = 0.001
niters = 1500
nsamples = 100
ntrain = 10000
ntest = 10000

model = 'linear'
dataset = 'PCA'

title =  f'(a) {model.capitalize()} VAE on {dataset} data set'

nmax = 1500
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=[5.25, 4])
    
fig.suptitle(title, fontsize=16)

output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain={ntrain}, ntest={ntest}, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}/'

suffix = [''] + [f" ({i})" for i in range(9)]
bounds = []
three_entropies = []
log_likelihood = []
test_log_likelihood = []
for s in suffix:
    output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain=10000, ntest=10000, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}{s}/'
    bound, three_entropy, ll, test_ll, ground_truth_ll = load_data(output_folder)
    bounds.append(bound)
    three_entropies.append(three_entropy)
    log_likelihood.append(ll)
    test_log_likelihood.append(test_ll)
ground_truth_ll =  np.asarray(ground_truth_ll)/ntrain
bounds = np.asarray(bounds)/ntrain
three_entropies = np.asarray(three_entropies)/ntrain
log_likelihood = np.asarray(log_likelihood)/ntrain
test_log_likelihood = np.asarray(test_log_likelihood)/ntest

irun = 1
niter = range(1,nmax+1)
ax1.plot(niter, log_likelihood[irun,:nmax], label='loglikelihood', color='tab:red', zorder=1.2)
ax1.plot(niter, test_log_likelihood[irun,:nmax], '--', label='held-out logl.', color='tab:orange', zorder=1.3)
ax1.axhline(ground_truth_ll, linestyle=':', color='black', label='ground-truth', zorder=1.5)
ax1.plot(niter, bounds[irun,:nmax], label='lower bound', color='lightblue', zorder=1)
ax1.plot(niter, three_entropies[irun,:nmax], label='three entropies', color='tab:green', zorder=1.4)

# sigma annotations
try:
    fix_sigma = 0.5
    output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain=10000, ntest=10000, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}, fix_sigma={fix_sigma}/'
    fs_bound, fs_three_entropies, *_ = load_data(output_folder)
    fs_bound = np.asarray(fs_bound)/ntrain
    fs_three_entropies = np.asarray(fs_three_entropies)/ntrain
    ax1.plot(niter, fs_bound[:nmax], '--', color='lightblue', zorder=0, alpha=0.7)
    ax1.plot(niter, fs_three_entropies[:nmax], '--', color='tab:green', zorder=0, alpha=0.7)
    ax1.text(1200, (fs_three_entropies[-1] + fs_bound[-1])/2*1.03, "$\sigma_\mathrm{fixed}$", fontsize=10, color='dimgrey', verticalalignment='center')
    ax1.plot([1150, 1200], [fs_three_entropies[-1], (fs_three_entropies[-1] + fs_bound[-1])/2], '-', color='tab:green', linewidth=1)
    ax1.plot([1200, 1150], [(fs_three_entropies[-1] + fs_bound[-1])/2, fs_bound[-1]], '-', color='lightblue', linewidth=1)
except:
    pass

diff = np.abs((bounds-three_entropies)/bounds[:,nmax-1][:,None])*100
perc = np.percentile(diff, [25 , 50, 75], axis=0)
ax2.plot(niter, perc[1,:nmax], label="median")
ax2.fill_between(niter, perc[0,:nmax], perc[2,:nmax], alpha=0.3)

ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel(r'$\frac{|\mathrm{LB} - \mathrm{3E}|}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)

ax1.zorder = 10
ax1.grid(linestyle=':', linewidth=0.4)
ax2.grid(linestyle=':', linewidth=0.4)

ax1.set_xlim(0, nmax)
#ax1.set_ylim(-25, 5)

ax2.set_xlim(0, nmax)
ax2.set_ylim(0, 4)

fig.tight_layout() 
fig.subplots_adjust(top=0.91)

handles, labels = ax1.get_legend_handles_labels()

green_line = Line2D([0,1], [1,1], color='tab:green', linestyle='-.')
blue_line = Line2D([0,0.5], [-1,-1], color='lightblue', linestyle='-.')
handles += [(green_line, blue_line)]
labels += [r'with fixed $\sigma$']

ax1.legend(handles, labels, loc="lower right", ncol=2, prop={'size': 10}, handler_map={tuple: HandlerTuple(ndivide=2)});

handles, labels = ax2.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["50% range"]

ax2.legend(handles, labels, loc="upper right", ncol=1, prop={'size': 10});

NameError: name 'plt' is not defined

## Fig. 1b: Non-linear VAE on MNIST

In [4]:
batch_size = 2000
lr = 0.001
niters = 200
nsamples = 100
ntrain = 60000
ntest = 10000

model = 'non-linear'
dataset = 'MNIST'


suffix = [''] + [f" ({i})" for i in range(9)]
bounds = []
three_entopies = []

for s in suffix:
    output_folder = f'./output/{model}/{dataset}/H=2, D=784, ntrain={ntrain}, ntest={ntest}, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}{s}/'
    bound, three_entropy, *_ = load_data(output_folder)
    bounds.append(bound)
    three_entopies.append(three_entropy)
bounds = np.asarray(bounds)/ntrain
three_entopies = np.asarray(three_entopies)/ntrain
    
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=[5.25, 4])
    
fig.suptitle('(b) VAE-1 on MNIST data set', fontsize=16)

nmax = 200
niter = range(1, nmax+1)
ax1.plot(niter, bounds[0,:nmax], label='lower bound', color='lightblue', zorder=1)
ax1.plot(niter, three_entopies[0,:nmax], label='three entropies', color='tab:green', zorder=2)
plot_i = [1,2]
for i in (plot_i):
    ax1.plot(niter, bounds[i,:nmax], color='lightblue', zorder=1)
    ax1.plot(niter, three_entopies[i,:nmax], color='tab:green', zorder=2)

diff = np.abs((bounds-three_entopies)/bounds[:,nmax-1][:,None])*100
perc = np.percentile(diff, [25 , 50, 75], axis=0)
ax2.plot(niter, perc[1,:nmax], label="median")
ax2.fill_between(niter, perc[0,:nmax], perc[2,:nmax], alpha=0.3)

ax1.zorder = 10
ax1.grid(linestyle=':', linewidth=0.4)
ax2.grid(linestyle=':', linewidth=0.4)

ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel(r'$\frac{|\mathrm{LB} - \mathrm{3E}|}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)

ax1.set_xlim(0, nmax)
ax2.set_xlim(0, nmax)

ax1.set_ylim(-200, 200)
ax2.set_ylim(0, 4)

fig.tight_layout() 
fig.subplots_adjust(top=0.91)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc="lower right", ncol=3, prop={'size': 10});

handles, labels = ax2.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["50% range"]

ax2.legend(handles, labels, loc="upper right", ncol=1, prop={'size': 10});

NameError: name 'np' is not defined

## Fig. 1c: Sigma-z-full VAE on non-linear PCA

In [5]:
batch_size = 2000
lr = 0.001
niters = 3000
nsamples = 100
ntrain = 10000
ntest = 10000

model = 'sigma-z-full'
dataset = 'PCA-ring'

suffix = [''] + [f" ({i})" for i in range(99)]
bounds = []
three_entopies = []

for s in suffix:
    output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain={ntrain}, ntest={ntest}, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}{s}/'
    bound, three_entropy, *_ = load_data(output_folder)
    bounds.append(bound)
    three_entopies.append(three_entropy)
bounds = np.asarray(bounds)/ntrain
three_entopies = np.asarray(three_entopies)/ntrain
    
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=[5.25, 4])
    
fig.suptitle('(c) VAE-3 on PCA-ring data set', fontsize=16)

nmax = 300
niter = range(1, nmax+1)
ax1.plot(niter, bounds[0,:nmax], label='lower bound', color='lightblue', zorder=1)
ax1.plot(niter, three_entopies[0,:nmax], label='three entropies', color='tab:green', zorder=2)
plot_i = [1,5]
for i in (plot_i):
    ax1.plot(niter, bounds[i,:nmax], color='lightblue', zorder=1)
    ax1.plot(niter, three_entopies[i,:nmax], color='tab:green', zorder=2)

diff = np.abs((bounds-three_entopies)/bounds[:,nmax-1][:,None])*100
perc = np.percentile(diff, [25 , 50, 75], axis=0)
ax2.plot(niter, perc[1,:nmax], label="median")
ax2.fill_between(niter, perc[0,:nmax], perc[2,:nmax], alpha=0.3)

ax1.grid(linestyle=':', linewidth=0.4)
ax2.grid(linestyle=':', linewidth=0.4)

ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel(r'$\frac{|\mathrm{LB} - \mathrm{3E}|}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)

ax1.set_xlim(0, nmax)
ax2.set_xlim(0, nmax)

ax1.set_ylim(-10, 10)
ax2.set_ylim(0, 4)

ax1.set_zorder(10)

fig.tight_layout() 
fig.subplots_adjust(top=0.91)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc="lower right", ncol=3, prop={'size': 10});

handles, labels = ax2.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["50% range"]

ax2.legend(handles, labels, loc="upper right", ncol=1, prop={'size': 10});

NameError: name 'np' is not defined

## Fig. 2 (Appendix)

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
batch_size = 2000
lr = 0.001
niters = 3000
nsamples = 100
ntrain = 10000
ntest = 10000

model = 'sigma-z-full'
dataset = 'PCA-ring'

suffix = [''] + [f" ({i})" for i in range(99)]
bounds = []
three_entropies = []

for s in suffix:
    output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain={ntrain}, ntest={ntest}, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}{s}/'
    bound, three_entropy, *_ = load_data(output_folder)
    bounds.append(bound)
    three_entropies.append(three_entropy)
bounds = np.asarray(bounds)/ntrain
three_entropies = np.asarray(three_entropies)/ntrain
    
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, gridspec_kw={'height_ratios': [2, 1, 1.3]}, figsize=[5.25, 6.5])
    
fig.suptitle('(a) VAE-3 on PCA-ring data set', fontsize=16)

nmax = 3000
niter = range(1, nmax+1)
ax1.plot(niter, bounds[0,:nmax], label='lower bound', color='lightblue', zorder=2)
ax1.plot(niter, three_entropies[0,:nmax], label='three entropies', color='tab:green', zorder=1)
plot_i = [5,7]
for i in (plot_i):
    ax1.plot(niter, bounds[i,:nmax], color='lightblue', zorder=2)
    ax1.plot(niter, three_entropies[i,:nmax], color='tab:green', zorder=1)

abs_diff = np.abs((bounds-three_entropies)/bounds[:,nmax-1][:,None])*100
abs_diff_perc = np.percentile(abs_diff, [25 , 50, 75], axis=0)
ax2.plot(niter, abs_diff_perc[1,:nmax], label="median")
ax2.fill_between(niter, abs_diff_perc[0,:nmax], abs_diff_perc[2,:nmax], alpha=0.3)

naverage = 100
mean = np.mean((bounds[:,:nmax] - three_entropies[:,:nmax])/bounds[:,:nmax], axis=0)*100
mov_mean = moving_average(mean, naverage)
std = np.std((bounds[:,:nmax] - three_entropies[:,:nmax])/bounds[:,:nmax], axis=0)*100#/np.sqrt(bounds.shape[0])
mov_std = moving_average(std, naverage)
ax3.plot(niter, mean, label="mean rel. deviation", zorder=2)
ax3.plot(niter[naverage//2-1:-naverage//2], mov_mean, label="moving average", linewidth=2, zorder=3)
ax3.fill_between(niter, mean-std, mean+std, alpha=0.3, zorder=1)

ax3.axhline(y=0, color='black')

ax1.grid(linestyle=':', linewidth=0.4)
ax2.grid(linestyle=':', linewidth=0.4)
ax3.grid(linestyle=':', linewidth=0.4)

ax2.set_title(r'rel. abs. error', fontsize=12)
ax2.set_ylabel(r'$\frac{|\mathrm{LB} - \mathrm{3E}|}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)

ax3.set_ylabel(r'$\frac{\mathrm{LB} - \mathrm{3E}}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)
ax3.set_title(r'rel. deviation', fontsize=12)
ax3.set_xlabel('Epoch', fontsize=12)

ax1.set_xlim(0, nmax)
ax2.set_xlim(0, nmax)
ax3.set_xlim(0, nmax)

ax1.set_ylim(5, 10)
ax2.set_ylim(0, 4)
ax3.set_ylim(-2, 4)

ax2.grid(True, which='major',linewidth=0.5)

ax1.set_zorder(10)

# final values on axes
ax21 = ax2.twinx()
ax21.set_ylim(ax2.get_ylim())
ax21.set_yticks([np.round(abs_diff_perc[1,-1], 2)])
ax21.tick_params(labelsize=8, pad=1)

ax31 = ax3.twinx()
ax31.set_ylim(ax3.get_ylim())
ax31.set_yticks([np.round(np.mean(mov_mean[-100:]),2)])
ax31.tick_params(labelsize=8, pad=1)


fig.tight_layout() 
fig.subplots_adjust(top=0.91)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc="lower right", ncol=3, prop={'size': 10});

handles, labels = ax2.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["50% range"]
ax2.legend(handles, labels, loc="upper right", ncol=1, prop={'size': 10});

handles, labels = ax3.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["std"]
ax3.legend(handles, labels, loc="upper right", ncol=2, prop={'size': 10});

In [ ]:
batch_size = 100
lr = 0.001
niters = 3000
nsamples = 100
ntrain = 10000
ntest = 10000

model = 'sigma-z-full'
dataset = 'PCA-ring'

suffix = [''] + [f" ({i})" for i in range(99)]
bounds = []
three_entopies = []

for s in suffix:
    output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain={ntrain}, ntest={ntest}, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}{s}/'
    try:
        bound, three_entropy, *_ = load_data(output_folder)
    except Exception as e:
        print(e)
        continue
    bounds.append(bound)
    three_entopies.append(three_entropy)
bounds = np.asarray(bounds)/ntrain
three_entopies = np.asarray(three_entopies)/ntrain
    
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, gridspec_kw={'height_ratios': [2, 1, 1.3]}, figsize=[5.25, 6.5])
    
fig.suptitle('(b) with smaller batch size', fontsize=16)

nmax = 3000
niter = range(1, nmax+1)
ax1.plot(niter, bounds[0,:nmax], label='lower bound', color='lightblue', zorder=2)
ax1.plot(niter, three_entopies[0,:nmax], label='three entropies', color='tab:green', zorder=1)
plot_i = [1,8]
for i in (plot_i):
    ax1.plot(niter, bounds[i,:nmax], color='lightblue', zorder=2)
    ax1.plot(niter, three_entopies[i,:nmax], color='tab:green', zorder=1)

abs_diff = np.abs((bounds-three_entopies)/bounds[:,nmax-1][:,None])*100
abs_diff_perc = np.percentile(abs_diff, [25 , 50, 75], axis=0)
ax2.plot(niter, abs_diff_perc[1,:nmax], label="median")
ax2.fill_between(niter, abs_diff_perc[0,:nmax], abs_diff_perc[2,:nmax], alpha=0.3)

naverage = 100
mean = np.mean((bounds[:,:nmax] - three_entopies[:,:nmax])/bounds[:,:nmax], axis=0)*100
mov_mean = moving_average(mean, naverage)
std = np.std((bounds[:,:nmax] - three_entopies[:,:nmax])/bounds[:,:nmax], axis=0)*100#/np.sqrt(bounds.shape[0])
mov_std = moving_average(std, naverage)
ax3.plot(niter, mean, label="mean rel. deviation", zorder=2)
ax3.plot(niter[naverage//2-1:-naverage//2], mov_mean, label="moving average", linewidth=2, zorder=3)
ax3.fill_between(niter, mean-std, mean+std, alpha=0.3, zorder=1)

ax3.axhline(y=0, color='black')

ax1.grid(linestyle=':', linewidth=0.4)
ax2.grid(linestyle=':', linewidth=0.4)
ax3.grid(linestyle=':', linewidth=0.4)


ax2.set_title(r'rel. abs. error', fontsize=12)
ax2.set_ylabel(r'$\frac{|\mathrm{LB} - \mathrm{3E}|}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)

ax3.set_ylabel(r'$\frac{\mathrm{LB} - \mathrm{3E}}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)
ax3.set_title(r'rel. deviation', fontsize=12)
ax3.set_xlabel('Epoch', fontsize=12)

ax1.set_xlim(0, nmax)
ax2.set_xlim(0, nmax)
ax3.set_xlim(0, nmax)

ax1.set_ylim(5, 10)
ax2.set_ylim(0, 4)
ax3.set_ylim(-2, 4)

ax1.set_zorder(10)

# final values on axes
ax21 = ax2.twinx()
ax21.set_ylim(ax2.get_ylim())
ax21.set_yticks([np.round(np.mean(abs_diff_perc[1,-100:]),2)])
ax21.tick_params(labelsize=8, pad=1)

ax31 = ax3.twinx()
ax31.set_ylim(ax3.get_ylim())
ax31.set_yticks([np.round(np.mean(mov_mean[-100:]),2)])
ax31.tick_params(labelsize=8, pad=1)


fig.tight_layout() 
fig.subplots_adjust(top=0.91)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc="lower right", ncol=3, prop={'size': 10});

handles, labels = ax2.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["50% range"]
ax2.legend(handles, labels, loc="upper right", ncol=1, prop={'size': 10});

handles, labels = ax3.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["std"]
ax3.legend(handles, labels, loc="upper right", ncol=2, prop={'size': 10});

In [ ]:
batch_size = 2000
lr = 0.005
niters = 3000
nsamples = 100
ntrain = 10000
ntest = 10000

model = 'sigma-z-full'
dataset = 'PCA-ring'

suffix = [''] + [f" ({i})" for i in range(99)]
bounds = []
three_entopies = []

for s in suffix:
    output_folder = f'./output/{model}/{dataset}/H=2, D=10, ntrain={ntrain}, ntest={ntest}, batch_size={batch_size}, lr={lr}, niters={niters}, nsamples={nsamples}{s}/'
    try:
        bound, three_entropy, *_ = load_data(output_folder)
    except Exception as e:
        print(e)
        break
    bounds.append(bound)
    three_entopies.append(three_entropy)
bounds = np.asarray(bounds)/ntrain
three_entopies = np.asarray(three_entopies)/ntrain
    
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, gridspec_kw={'height_ratios': [2, 1, 1.3]}, figsize=[5.25, 6.5])
    
fig.suptitle('(c) with higher learning rate', fontsize=16)

nmax = 3000
niter = range(1, nmax+1)
ax1.plot(niter, bounds[0,:nmax], label='lower bound', color='lightblue', zorder=2)
ax1.plot(niter, three_entopies[0,:nmax], label='three entropies', color='tab:green', zorder=1)
plot_i = [1,9]
for i in (plot_i):
    ax1.plot(niter, bounds[i,:nmax], color='lightblue', zorder=2)
    ax1.plot(niter, three_entopies[i,:nmax], color='tab:green', zorder=1)

abs_diff = np.abs((bounds-three_entopies)/bounds[:,nmax-1][:,None])*100
abs_diff_perc = np.percentile(abs_diff, [25 , 50, 75], axis=0)
ax2.plot(niter, abs_diff_perc[1,:nmax], label="median")
ax2.fill_between(niter, abs_diff_perc[0,:nmax], abs_diff_perc[2,:nmax], alpha=0.3)

naverage = 100
mean = np.mean((bounds[:,:nmax] - three_entopies[:,:nmax])/bounds[:,:nmax], axis=0)*100
mov_mean = moving_average(mean, naverage)
std = np.std((bounds[:,:nmax] - three_entopies[:,:nmax])/bounds[:,:nmax], axis=0)*100#/np.sqrt(bounds.shape[0])
mov_std = moving_average(std, naverage)
ax3.plot(niter, mean, label="mean rel. deviation", zorder=2)
ax3.plot(niter[naverage//2-1:-naverage//2], mov_mean, label="moving average", linewidth=2, zorder=3)
ax3.fill_between(niter, mean-std, mean+std, alpha=0.3, zorder=1)

ax3.axhline(y=0, color='black')

ax1.grid(linestyle=':', linewidth=0.4)
ax2.grid(linestyle=':', linewidth=0.4)
ax3.grid(linestyle=':', linewidth=0.4)

ax2.set_title(r'rel. abs. error', fontsize=12)
ax2.set_ylabel(r'$\frac{|\mathrm{LB} - \mathrm{3E}|}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)

ax3.set_ylabel(r'$\frac{\mathrm{LB} - \mathrm{3E}}{|\mathrm{LB}_\mathrm{final}|}$  [%]', fontsize=12)
ax3.set_title(r'rel. deviation', fontsize=12)
ax3.set_xlabel('Epoch', fontsize=12)

ax1.set_xlim(0, nmax)
ax2.set_xlim(0, nmax)
ax3.set_xlim(0, nmax)

ax1.set_ylim(5, 10)
ax2.set_ylim(0, 4)
ax3.set_ylim(-2, 4)

ax1.set_zorder(10)

# final values on axes
ax21 = ax2.twinx()
ax21.set_ylim(ax2.get_ylim())
ax21.set_yticks([np.round(np.mean(abs_diff_perc[1,-100:]),2)])
ax21.tick_params(labelsize=8, pad=1)

ax31 = ax3.twinx()
ax31.set_ylim(ax3.get_ylim())
ax31.set_yticks([np.round(np.mean(mov_mean[-100:]),2)])
ax31.tick_params(labelsize=8, pad=1)


fig.tight_layout() 
fig.subplots_adjust(top=0.91)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc="lower right", ncol=3, prop={'size': 10});

handles, labels = ax2.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["50% range"]
ax2.legend(handles, labels, loc="upper right", ncol=1, prop={'size': 10});

handles, labels = ax3.get_legend_handles_labels()
blue_rectangle = Rectangle((0.5, 0.5), 1, 1, facecolor='tab:blue', edgecolor='None', alpha=0.3, fill=True)
handles += [blue_rectangle]
labels += ["std"]
ax3.legend(handles, labels, loc="upper right", ncol=2, prop={'size': 10});
